In [19]:
import boto3
import pandas as pd
import numpy as np
from sklearn.linear_model import LogisticRegression, Lasso, LassoCV
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import recall_score

## Defining the bucket 
s3 = boto3.resource('s3')
bucket_name = 'data-445'
bucket = s3.Bucket(bucket_name)

## Defining the csv file 
file_key = 'Fall_2021/In_Class_Assignments/framingham.csv'

bucket_object = bucket.Object(file_key)
file_object = bucket_object.get()
file_content_stream = file_object.get('Body')

## Reading the csv file
heart = pd.read_csv(file_content_stream)
heart = heart.dropna()
heart.head()

,male,age,education,currentSmoker,cigsPerDay,BPMeds,prevalentStroke,prevalentHyp,diabetes,totChol,sysBP,diaBP,BMI,heartRate,glucose,TenYearCHD
0,1,39,4.0,0,0.0,0.0,0,0,0,195.0,106.0,70.0,26.97,80.0,77.0,0
1,0,46,2.0,0,0.0,0.0,0,0,0,250.0,121.0,81.0,28.73,95.0,76.0,0
2,1,48,1.0,1,20.0,0.0,0,0,0,245.0,127.5,80.0,25.34,75.0,70.0,0
3,0,61,3.0,1,30.0,0.0,0,1,0,225.0,150.0,95.0,28.58,65.0,103.0,1
4,0,46,3.0,1,23.0,0.0,0,0,0,285.0,130.0,84.0,23.10,85.0,85.0,0


In [5]:
heart['TenYearCHD'].value_counts() / heart.shape[0]

0    0.847648
1    0.152352
Name: TenYearCHD, dtype: float64

In [15]:
heart['male'].value_counts() / heart.shape[0]

0    0.556346
1    0.443654
Name: male, dtype: float64

In [17]:
## Defining the input and target variables
X = heart[['male', 'age', 'currentSmoker', 'totChol', 'sysBP', 'BMI', 'heartRate', 'glucose']] 
Y = heart['TenYearCHD']

## Splitting the data 
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size = 0.2, stratify = Y)

## Transforming the inputs to 0-1 scale
scaler = MinMaxScaler()
X_train = pd.DataFrame(scaler.fit_transform(X_train), columns = X_train.columns)
X_test = pd.DataFrame(scaler.fit_transform(X_test), colums = X_test.columns)

In [22]:
## Running lasso cv
lasso_cv = LassoCV(cv = 5).fit(X_train, Y_train)
lasso_cv.alpha_

## Selecting important features from LASSO
lasso_md = Lasso(alpha = lasso_cv.alpha_).fit(X_train, Y_train)
lasso_md.coef_

array([ 5.63853721e-02,  2.72939945e-01,  4.08018244e-02,  2.01544301e-04,
        5.46736858e-01,  0.00000000e+00, -2.76377828e-02,  3.01462568e-01])

In [23]:
X_train_logit = X_train[['male', 'age', 'currentSmoker', 'sysBP', 'glucose']]
X_test_logit = X_test[['male', 'age', 'currentSmoker', 'sysBP', 'glucose']]

/home/ec2-user/anaconda3/envs/python3/lib/python3.6/site-packages/ipykernel/__main__.py:1: FutureWarning: Using a non-tuple sequence for multidimensional indexing is deprecated; use `arr[tuple(seq)]` instead of `arr[seq]`. In the future this will be interpreted as an array index, `arr[np.array(seq)]`, which will result either in an error or a different result.
  if __name__ == '__main__':


IndexError: only integers, slices (`:`), ellipsis (`...`), numpy.newaxis (`None`) and integer or boolean arrays are valid indices

In [18]:
## Fitting the random forest classifier
md = RandomForestClassifier(n_estimators = 500).fit(X_train, Y_train)

## Selecting important features from random forest 
md.feature_importances_

array([0.02207902, 0.15115329, 0.02241045, 0.16042334, 0.19246142,
       0.17509623, 0.12424728, 0.15212897])

In [11]:
## Logistic regression 
md1 = LogisticRegression().fit(X_train, Y_train)

## Predicting on the test dataset
pred1 = md1.predict_proba(X_test)[:, 1]
pred1 = np.where(pred1 < 0.10, 0, 1)

## Computing the recall-score
recall_score(Y_test, pred1)

0.8571428571428571

In [12]:
## Random forest
md2 = RandomForestClassifier(n_estimators = 500, max_depth = 5).fit(X_train, Y_train)

## Predicting on the test dataset 
pred2 = md2.predict_proba(X_test)[:, 1]
pred2 = np.where(pred2 < 0.10, 0, 1)

## Computing the recall-score
recall_score(Y_test, pred2)

0.9107142857142857